In [1]:
!pip uninstall ragatouille -y

Found existing installation: RAGatouille 0.0.8.post4
Uninstalling RAGatouille-0.0.8.post4:
  Successfully uninstalled RAGatouille-0.0.8.post4


In [2]:
!pip install lancedb --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.2/32.2 MB 46.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 51.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pylance
    Found existing installation: pylance 0.19.2
    Uninstalling pylance-0.19.2:
      Successfully uninstalled pylance-0.19.2
  Attempting uninstall: lancedb
    Found existing installation: lancedb 0.16.0
    Uninstalling lancedb-0.16.0:
      Successfully uninstalled lancedb-0.16.0


In [3]:
!pip install langchain --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.9 MB/s eta 0:00:00 0:00:01
  Using cached langchain_text_splitters-0.3.5-py3-none-any.whl (31 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 KB 80.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.43
    Uninstalling langchain-core-0.2.43:
      Successfully uninstalled langchain-core-0.2.43
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.2.4
    Uninstalling langchain-text-splitters-0.2.4:
      Successfully uninstalled langchain-text-splitters-0.2.4
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.6
    Uninstalling langchain-0.2.6:
      Successfully uninstalled langchain-0.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-q

In [4]:
!pip install langchain-community --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.7 MB/s eta 0:00:0000:010:01
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.2.6
    Uninstalling langchain-community-0.2.6:
      Successfully uninstalled langchain-community-0.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.0.62 requires langchain-community<0.3.0,>=0.2.6, but you have langchain-community 0.3.15 which is incompatible.
langchain-experimental 0.0.62 requires langchain-core<0.3.0,>=0.2.10, but you have langchain-core 0.3.31 which is incompatible.


In [5]:
!pip install langchain-huggingface --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 KB 3.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.5.1
    Uninstalling sentence-transformers-2.5.1:
      Successfully uninstalled sentence-transformers-2.5.1


In [6]:
!pip install tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 111.3 MB/s eta 0:00:0000:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0


In [50]:
import os
import json
import pickle
from textwrap import dedent
from typing import List, Tuple
from abc import ABC, abstractmethod
from collections.abc import Iterable, AsyncIterable, Callable

from tqdm.notebook import tqdm
from IPython.display import clear_output, display, HTML, Markdown

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer, pipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import LanceDB
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_core.language_models.llms import BaseLLM
from langchain_core.retrievers import BaseRetriever
from langchain_core.vectorstores.base import VectorStore
import lancedb
import numpy as np
import pandas as pd
import torch


In [8]:
ARTEFACT_VERSION = '03'

In [9]:
ARTEFACT_ROOT_FOLDER = os.environ.get('ARTEFACT_ROOT_FOLDER', '/artefact')
ARTEFACT_FOLDER = os.path.join(ARTEFACT_ROOT_FOLDER, 'eberron', f'v{ARTEFACT_VERSION}')

In [10]:
HF_HOME = os.environ.get('HF_HOME')

In [11]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 3 MiB, 15093 MiB


# Load the Artefact

In [12]:
try:
    with open(os.path.join(ARTEFACT_FOLDER, 'model_metadata.json'), 'r') as f:
        model_metadata = json.load(f)
except FileNotFoundError:
    with open(os.path.join(ARTEFACT_FOLDER, 'model_metadata.pkl'), 'rb') as f:
        model_metadata = pickle.load(f)

assert model_metadata['embedding_model']['str'].startswith('SentenceTransformer')

In [13]:
if model_metadata['embedding_format'] == 'pickle':
    with open(os.path.join(ARTEFACT_FOLDER, 'embeddings.pkl'), 'rb') as f:
        embeddings = pickle.load(f)
elif model_metadata['embedding_format'] == 'lancedb':
    embeddings_folder = os.path.join(ARTEFACT_FOLDER, 'embeddings')
    db = lancedb.connect(embeddings_folder)
    table = db.open_table('documents')

In [14]:
if model_metadata['version'] == '01':
    # I used a JSON to store metadata only in artefact v01. v02 onwards, they are in lancedb fields
    with open(os.path.join(ARTEFACT_FOLDER, 'chunk_metadata.json'), 'r') as f:
        chunk_metadata = json.load(f)


In [15]:
if model_metadata['version'] == '01':
    file_names = [f for f in os.listdir(os.path.join(ARTEFACT_FOLDER, 'chunks')) if f.endswith('.md')]
    file_names = sorted(file_names)
    chunks = [None] * len(file_names)
    for file_name in tqdm(file_names):
        file_path = os.path.join(ARTEFACT_FOLDER, 'chunks', file_name)
        with open(file_path, 'r') as f:
            chunks[int(file_name.split('.')[0])] = f.read()

# Load the Embedding Model

In [16]:
# embedding_model = SentenceTransformer(model_metadata['embedding_model']['name'], 
#                                       trust_remote_code=True, 
#                                       revision=model_metadata['embedding_model']['revision'])
# embedding_model = embedding_model.to("cpu")
# # embedding_model = embedding_model.to("cuda")
embeddings = HuggingFaceEmbeddings(model_name=model_metadata['embedding_model']['name'], 
                                   model_kwargs={'device': 'cpu', 
                                                 'revision': model_metadata['embedding_model']['revision'],
                                                 'trust_remote_code': True},
                                   encode_kwargs={'normalize_embeddings': True})

/tmp/ipykernel_67/2386398130.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_metadata['embedding_model']['name'],


# Load the LLM

In [17]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 3 MiB, 15093 MiB


In [18]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


In [19]:
model_provider = 'mistralai'
model_name = 'Mistral-7B-Instruct-v0.3'
model_revision = 'e0bc86c23ce5aae1db576c8cca6f06f1f73af2db'
model_id = f'{model_provider}/{model_name}'
model_path = os.path.join(HF_HOME, 'hub', f'models--{model_provider}--{model_name}', 'snapshots', model_revision)
if os.path.exists(model_path):
    print("Found model in cache, reading from the cache")
    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, quantization_config=bnb_config, device_map="auto")
else:
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, quantization_config=bnb_config, device_map="auto")

Found model in cache, reading from the cache


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [21]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 4087 MiB, 11009 MiB


# Instantiate Lang* Objects

In [22]:
vector_store = LanceDB(connection=db, table_name='documents', embedding=embeddings)

In [23]:
# similarity_retriever_top_5 = vector_store.as_retriever(search_type="similarity", search_kwargs={'k': 5})
# similarity_retriever_top_7 = vector_store.as_retriever(search_type="similarity", search_kwargs={'k': 7})
# similarity_retriever_top_10 = vector_store.as_retriever(search_type="similarity", search_kwargs={'k': 10})

In [24]:
# # Test
# docs = similarity_retriever_top_5.invoke("Tell me about the languages of Eberron.")
# assert len(docs) == 5

In [24]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 4087 MiB, 11009 MiB


In [25]:
# type(content_summary_pipeline).__bases__

In [26]:
# type(content_summary_retriever).__bases__

In [27]:
# type(vector_store).__bases__

In [28]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 4087 MiB, 11009 MiB


In [198]:
class Agent(ABC):
    """
    Abstract base class for an AI agent that interacts with a language model
    and retrieves information to respond to queries.
    """

    base_llm: BaseLLM
    """
    The base language model used by the agent. This model is responsible for
    generating responses and performing language processing tasks.
    """
    
    q_and_a: List[Tuple[str, str | Callable]]
    """
    A list of tuples, where each tuple contains:
        - A question (str).
        - The expected response, which can either be:
            - A string (str), or
            - A callable function (Callable) that returns the expected response as a string.
    Used for evaluating the agent's performance.
    """

    @property
    @abstractmethod
    def version(self) -> str:
        """
        Abstract property to return the version of the agent.
        
        Returns:
            str: The version of the agent.
        """
        return ""
    
    @abstractmethod
    def listen(self, context: str) -> bool:
        """
        Other agents can call this function to pass contextual information.

        Args:
            context (str): The contextual information in natural language.

        Returns:
            bool: Return True if the knowledge has been accepted
        """
        return True

    @abstractmethod
    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        """
        Retrieve relevant information for a given query.

        Args:
            q (str): The query string for which relevant data needs to be retrieved.

        Returns:
            List[Tuple[str, dict]]: A list of tuples where each tuple contains
            a relevant piece of information (e.g., a document or text snippet) as a string,
            and its associated metadata as a dictionary.
        """
        return []

    @abstractmethod
    def _prompt(self, q: str) -> str:
        """
        Generate a formatted prompt based on the given query.

        Args:
            q (str): The query string for which the prompt is generated.

        Returns:
            str: The formatted prompt that will be used for generating a response.
        """
        return ""

    @abstractmethod
    async def respond(self, q: str) -> AsyncIterable[str]:
        """
        Generate a response for the given query by retrieving information
        and interacting with the language model.

        Args:
            q (str): The query string for which the response is generated.

        Yields:
            Iterable[str]: An iterable of response strings generated by the agent.
        """
        while False:
            yield ""

    async def ask(self, agent: "Agent", q: str) -> AsyncIterable[str]:
        """
        Asynchronously send a query to another agent and process its response.

        Args:
            agent (Agent): Another agent to which the query is sent.
            q (str): The query string to be sent.

        Yields:
            Iterable[str]: An iterable of response strings generated by the other agent.
        """
        self.received_response = ""
        async for chunk in agent.respond(q):
            self.received_response += chunk
            yield chunk
            
    async def receive(self, generator: AsyncIterable) -> str:
        return ''.join([c async for c in generator])

    def tell(self, agent: "Agent", context: str) -> bool:
        """
        Send information to another agent.

        Args:
            agent (Agent): The recipient agent to which the information is sent.
            context (str): The information or context to be shared, framed as a query.

        """
        return agent.listen(context)
            
    async def evaluate(self) -> Tuple[float, np.ndarray]:
        """
        Evaluate the agent's performance by comparing its responses to a list of
        predefined question-answer pairs (`q_and_a`).

        This method iterates through the list of questions and their corresponding
        expected answers. For each question, the agent generates a response using
        the `respond` method. The response is compared to the expected answer
        (either a string or the output of a callable function) to determine correctness.

        Returns:
            Tuple[float, np.ndarray]: A tuple containing:
                - The accuracy as a float (correct answers / total questions), formatted to three decimal places.
                - A NumPy array (`np.ndarray`) where each element is 1 if the response matched the expected answer,
                  or 0 otherwise.

        Example:
            If `q_and_a` contains 10 questions and the agent answers 7 correctly,
            and their correctness is stored in a NumPy array, this method will return `(0.700, array([1, 1, 0, ...]))`.
        """
        matches = np.zeros(len(self.q_and_a))
        i = 0
        for q, expected_response in self.q_and_a:
            if isinstance(expected_response, str):
                if ''.join([r async for r in self.respond(q)]).strip() == expected_response.strip():
                    matches[i] = 1
            else:
                if ''.join([r async for r in self.respond(q)]).strip() == expected_response().strip():
                    matches[i] = 1
            i += 1
        
        return f'{matches.sum() / len(matches):.03f}', matches


In [199]:
class CanonicalSummaryAgent(Agent):
    version = '01'
    base_llm = HuggingFacePipeline(
        pipeline=pipeline(
            "text-generation", 
            model=model, 
            tokenizer=tokenizer, 
            temperature=0.1,
            max_new_tokens=768, 
            do_sample=True
        )
    )

    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information"""
        return False
    
    def __init__(self, vector_store: VectorStore, search_type: str, search_kwargs: dict):
        self.retriever = vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)

    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        self._last_retrieved_docs = self.retriever.invoke(q)

        return [(d.page_content, d.metadata) for d in self._last_retrieved_docs]

    def _prompt(self, q: str):
        retrieved_documents = [d[0] for d in self._retrieve(q)]

        retrieved_text = '\n\n'.join(retrieved_documents)
        
        self._last_prompt = dedent(f"""[INST]
        Use the following information (until the final cutoff =====) to answer the user query Q below.
        Prefer information closer to the top.
        
        {retrieved_text}
        
        =====
        
        Q:
        {q}
        
        A:
        [/INST]""")
        return self._last_prompt

    async def respond(self, q: str):
        # empty_chunk_count = 0
        self._last_response = ""
        prompt = self._prompt(q)
        for chunk in self.base_llm.bind(skip_prompt=True).stream(prompt):
            self._last_response += chunk
            yield chunk


Device set to use cuda:0


In [200]:
canonical_summary_agent = CanonicalSummaryAgent(vector_store, "similarity", {'k': 5})

In [72]:
response = ""
async for chunk in canonical_summary_agent.respond("What are the languages in Eberron"):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

In Eberron, the following languages are mentioned:

1. Abyssal: Common tongue of all fiends, also known as "Khyber’s Speech."
2. Argon: Spoken by barbarians of Argonnessen.
3. Auran: Air-based creatures like elves and dryads use this language.
4. Common: The language of the Five Nations and the language of trade in Khorvaire, known by most of its people.
5. Daan (Daanvi' and Daan of Kythri'): Used by Formians, Daelkyr, and Slaadi.
6. Daelkyr: Spoken by Daelkyr, mind flayers, and other Daelkyr aberrations.
7. Draconic: Used by kobolds, troglodytes, lizardfolk, dragons, ogres, giants, drow, and dragons.
8. Druidic: Used only by druids.
9. Elven: Used by elves and drow.
10. Giant: Used by ogres, giants, and drow.
11. Gnoll: Used by gnolls.
12. Goblin: Used by goblins, hobgoblins, and bugbears.
13. Halfling: Used by halflings.
14. Ignan: Used by fire-based creatures.
15. Infernal: Used by devils of Shavarath.
16. Irial: Used by Ravids, positive energy users.
17. Kythric: Used by Slaadi.
18. Mabran: Used by Nightshades, shadows, and creatures of Mabar.
19. Orc: Used by orcs.
20. Quori: Used by Quori and kalashtar.
21. Riedran: Used by the lower classes of Sarlona.
22. Sylvan: Used by dryads, eladrins, and creatures of Thelanis.
23. Terran: Used by Xorns and other earth-based creatures.
24. Undercommon: Used by chokers and underground Daelkyr denizens.

These languages have their own unique alphabets, scripts, and origins. Eberron is the language of the primal spirits of the world, and Druidic is derived from Eberral. Eberral is also the language of Lamannia and the natural world.

In [40]:
response = ""
async for chunk in canonical_summary_agent.respond("Tell me about the rivers of Khorvaire."):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

The information provided does not detail the rivers of Khorvaire specifically. However, it is mentioned that central Khorvaire remains a bastion of civilization, implying that it likely has developed infrastructure such as rivers for transportation and irrigation. The cities of Aundair, where dueling is still practiced despite recent laws, and Sharn, the focus of the Wayfinder's Guide, are both located in Khorvaire. These cities may have significant rivers or bodies of water. For more detailed information about the rivers of Khorvaire, further research or referencing the sources presented in the appendix would be necessary.

In [220]:
response = ""
async for chunk in canonical_summary_agent.respond("Tell me about fashion in the five nations."):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

The fashion in the Five Nations of Khorvaire, following the Last War, has evolved to reflect each nation's unique cultural identity. Common armor for soldiers includes light, medium, and heavy armor, with variations between nations. For example, Brelish splint mail combines a breastplate with a layer of chainmail, while Karrnathi splint mail is light plate.

Adventurers, an important part of each nation, often incorporate elements of their homeland's fashion into their attire, even if it isn't the most practical for the situation. Examples include gloves and masks for a Cyran, gazyrs sewn onto leather armor for a Karrn, a cloak with strange proportions for an Aundairian, clothes covered in embroidered patches for a Thrane, or pointy-toed shoes and long puffy sleeves for a Brelish.

Modern fashion designers now produce lines of fashion specifically for adventurers, recognizing their unique and often hard-to-pin-down personalities. Some adventurers may choose to wear specific colors or styles as a nod to their cultural roots, while others may not care about their home nation or religion and instead opt for more practical or personalized attire.

The Treaty of Thronehold, which divided the Five Nations into separate sovereign nations, has further solidified their cultural experiences, though some are too new or lawless to have a fully developed cultural identity. Pre-War Five Nations background grants equipment, skills, tools, and proficiencies specific to the nation you consider your homeland.

In [201]:
class ImprovisorAgent(Agent):
    version = '01'
    base_llm = HuggingFacePipeline(
        pipeline=pipeline(
            "text-generation", 
            model=model, 
            tokenizer=tokenizer, 
            temperature=0.7,
            max_new_tokens=768, 
            do_sample=True
        )
    )

    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information."""
        return False
    
    def __init__(self, vector_store: VectorStore, search_type: str, search_kwargs: dict):
        self.retriever = vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)

    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        self._last_retrieved_docs = self.retriever.invoke(q)

        return [(d.page_content, d.metadata) for d in self._last_retrieved_docs]

    def _prompt(self, q: str):
        retrieved_documents = [d[0] for d in self._retrieve(q)]

        retrieved_text = '\n\n'.join(retrieved_documents)
        
        self._last_prompt = dedent(f"""[INST]
        Use the following information as inspiration (until the final cutoff =====) to answer the user query Q below.
        Be creative.
        Come up with interesting information.
        
        
        {retrieved_text}
        
        =====
        
        Q:
        {q}
        
        A:
        [/INST]""")
        return self._last_prompt

    async def respond(self, q: str) -> AsyncIterable:
        # empty_chunk_count = 0
        prompt = self._prompt(q)
        self._last_response = ""
        for chunk in self.base_llm.bind(skip_prompt=True).stream(prompt):
            # if empty_chunk_count > 3:
            #     return ''
            # if chunk.strip() == '':
            #     empty_chunk_count += 1
            self._last_response += chunk
            yield chunk


Device set to use cuda:0


In [202]:
improvisor_agent = ImprovisorAgent(vector_store, "similarity", {'k': 5})

In [179]:
class CharacterPrompterAgent(Agent):
    version = '01'
    base_llm = HuggingFacePipeline(
        pipeline=pipeline(
            "text-generation", 
            model=model, 
            tokenizer=tokenizer, 
            temperature=0.9,
            max_new_tokens=512, 
            do_sample=True
        )
    )

    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information"""
        return False
    
    def __init__(self, vector_store: VectorStore, search_type: str, search_kwargs: dict):
        self.retriever = vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)

    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        self._last_retrieved_docs = self.retriever.invoke(q)

        return [(d.page_content, d.metadata) for d in self._last_retrieved_docs]

    def _prompt(self, q: str):
        retrieved_documents = [d[0] for d in self._retrieve(q)]

        retrieved_text = '\n\n'.join(retrieved_documents)
        
        self._last_prompt = dedent(f"""[INST]
        Use the following information as inspiration (until the final cutoff =====) to come up with a character concept that fits the user query Q.
        A character concept includes one paragraph on backstory, one paragraph on long-term goals, and one paragraph of immediate wants and needs.
        Sugggest a few persoanlity traits and secrets. 
        Come up with relevant D&D 5e skill checks for players to notice or figure out certain quirks or secrets of the character.
        
        
        {retrieved_text}
        
        =====
        
        Q:
        {q}
        
        A:
        [/INST]""")
        return self._last_prompt

    async def respond(self, q: str) -> AsyncIterable[str]:
        # empty_chunk_count = 0
        prompt = self._prompt(q)
        self._last_response = ""
        for chunk in self.base_llm.bind(skip_prompt=True).stream(prompt):
            # if empty_chunk_count > 3:
            #     return ''
            # if chunk.strip() == '':
            #     empty_chunk_count += 1
            self._last_response += chunk
            yield chunk


Device set to use cuda:0


In [180]:
character_prompter_agent = CharacterPrompterAgent(vector_store, "similarity", {'k': 10})

In [181]:
class CharacterClassFinderAgent(Agent):
    version = '01'
    base_llm = HuggingFacePipeline(
        pipeline=pipeline(
            "text-generation", 
            model=model, 
            tokenizer=tokenizer, 
            temperature=0.1,
            max_new_tokens=128, 
            do_sample=True
        )
    )

    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information"""
        return False
    
    def __init__(self, vector_store: VectorStore, search_type: str, search_kwargs: dict):
        self.retriever = vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)

    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        return []

    def _prompt(self, q: str):
        retrieved_documents = [d[0] for d in self._retrieve(q)]

        retrieved_text = '\n\n'.join(retrieved_documents)
        
        self._last_prompt = dedent(f"""[INST]
        Based on the prompt Q, what is a good D&D class for this character?
        Choose from the following list:
        Artificer, Bard, Barbarian, Fighter, Sorceror, Wizard, Rogue
        
        {retrieved_text}
        
        =====
        
        Q:
        {q}
        
        A:
        [/INST]""")
        return self._last_prompt

    async def respond(self, q: str) -> AsyncIterable[str]:
        # empty_chunk_count = 0
        prompt = self._prompt(q)
        self._last_response = ""
        for chunk in self.base_llm.bind(skip_prompt=True).stream(prompt):
            # if empty_chunk_count > 3:
            #     return ''
            # if chunk.strip() == '':
            #     empty_chunk_count += 1
            self._last_response += chunk
            yield chunk


Device set to use cuda:0


In [182]:
class CharacterGeneratorAgent(Agent):
    version = '01'
    base_llm = HuggingFacePipeline(
        pipeline=pipeline(
            "text-generation", 
            model=model, 
            tokenizer=tokenizer, 
            temperature=0.2,
            max_new_tokens=768, 
            do_sample=True
        )
    )

    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information"""
        return False
    
    # def __init__(self, vector_store: VectorStore, search_type: str, search_kwargs: dict):
    #     self.retriever = vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)

    def __init__(self):
        pass

    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        # TODO: Change this retrieve information based on the class...
        # self._last_retrieved_docs = self.retriever.invoke(q)

        # return [(d.page_content, d.metadata) for d in self._last_retrieved_docs]
        return []

    def _prompt(self, q: str):
        retrieved_documents = [d[0] for d in self._retrieve(q)]

        retrieved_text = '\n\n'.join(retrieved_documents)
        
        # Use the prompt P to create a complete character sheet based on the 5th Edition rules for the world of Eberron.
        # A complete character sheet should include name, race, class, level, attributes, proficiencies based on the class and fifth edition rules, 

        self._last_prompt = dedent(f"""[INST]
        Use the following character description to create a complete character sheet based on the 5th Edition rules for the world of Eberron.
        Make sure that the class fits the description, and make sure that the name is the same as in the description.

        Character Description:
        {q}
        ========

        Create the character sheet below.
        A complete character sheet should include race, class, level, attributes, proficiencies based on the class and fifth edition rules, 
        proficiency bonus based on class and level, saving throws bonus based on class, feats and proficiency , hit points based on class and level, 
        weapons and equipment, attack bonus for each weapon based on abilities, any race or class based features, class and level, attack roll, 
        and equipment, armor class based on dexterity and armor, and if the character has spellcasting ability, spells based on class and level, 
        as well as the number of spell slots. 
        Include an inventory based on their level, class, and race, with a few personal touches in the items.
        
        {retrieved_text}
        
        Character Sheet:
        [/INST]""")
        return self._last_prompt

    async def respond(self, q: str) -> AsyncIterable[str]:
        # empty_chunk_count = 0
        prompt = self._prompt(q)
        self._last_response = ""
        for chunk in self.base_llm.bind(skip_prompt=True).stream(prompt):
            self._last_response += chunk
            yield chunk


Device set to use cuda:0


In [183]:
character_generator_agent = CharacterGeneratorAgent()

In [184]:
class RequestClassifierAgent(Agent):
    version = '01'
    base_llm = HuggingFacePipeline(
        pipeline=pipeline(
            "text-generation", 
            model=model, 
            tokenizer=tokenizer, 
            temperature=0.1,
            max_new_tokens=64, 
            do_sample=True
        )
    )

    # TODO: Move this evaluation & classification to another agent.
    q_and_a = [
        ("Tell me about the languages of Eberron.", "lookup"),
        ("War veteran", "character"),
        ("Create a House Cannith item.", "original"),
        ("Create the details of a town on the border between Zilargo and Breland.", "original"),
        ("Groggy comic relief", "character"),
        ("Tell me about fashion in the five nations.", "lookup"),
    ]
    
    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information"""
        return False
    
    def __init__(self):
        pass

    def _retrieve(self, q: str) -> List[Tuple[str, dict]]:
        return []

    def _prompt(self, q: str):
        return dedent(f"""[INST]
        User is making a request. Classify the request into one of the three categories. Response with only one word.
        If the user is asking to create some original content, respond with the word "original".
        If this is a request to create a character or NPC, or it looks like it is decribing a D&D character, respond with the word "character".
        If the user is making a request to find out information, respond with the word "lookup".
        
        Q:
        {q}
        
        A:
        [/INST]""")

    async def respond(self, q: str) -> AsyncIterable[str]:
        if q.startswith("/character"):
            yield 'character'
            return
        
        category = self.base_llm.bind(skip_prompt=True).invoke(self._prompt(q))

        yield category.strip()


Device set to use cuda:0


In [167]:
request_classifier = RequestClassifierAgent()

In [168]:
await request_classifier.evaluate()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


('1.000', array([1., 1., 1., 1., 1., 1.]))

In [205]:
class SupervisorAgent(Agent):
    version = '01'
    base_llm = None

    def listen(self, context: str) -> bool:
        """This agent does not listen to any contextual information"""
        return False
    
    def _retrieve(self, q: str):
        """This agent does not retrieve any contextual information."""
        return []

    def _prompt(self, q: str):
        """This agent does not require a prompt."""
        return ""

    async def respond(self, q: str) -> AsyncIterable[str]:
        
        category_classification = self.ask(request_classifier, q)
        category = await self.receive(category_classification)
        
        if category == 'character':
            q = q.lstrip('/character')
            async for chunk in self.ask(character_prompter_agent, q):
                yield chunk
            async for chunk in self.ask(character_generator_agent, self.received_response):
                yield chunk
        elif category == 'lookup':
            q = q.lstrip('/lookup')
            async for chunk in self.ask(canonical_summary_agent, q):
                yield chunk
        elif category == 'original':
            q = q.lstrip('/original')
            async for chunk in self.ask(improvisor_agent, q):
                yield chunk
            
        

In [206]:
supervisor = SupervisorAgent()

In [207]:
response = ""
async for chunk in supervisor.respond("Create a Brelish war veteran"):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

Backstory:
Cameron Calamity Jones, a 42-year-old human male, was once an unremarkable private in the Brelish army during the last centurion-long war. Over the years of conflict, Jones rose through the ranks, becoming a skilled and respected soldier. His exceptional ability to cast spells quickly and accurately, even under intense pressure, earned him the moniker "Calamity." He was eventually promoted to the rank of Lieutenant, where he led a platoon of soldiers on the front lines.

Long-term Goals:
Jones' long-term goal is to rebuild Breland, using his skills and influence to help create a peaceful and prosperous future for the Five Nations. He aims to establish a network of individuals who share this vision, and together they will work to prevent another devastating war like the one just ended.

Immediate Wants and Needs:
Jones currently lives within the Tents of Quickstone, working as a consultant for the Grizzlies, offering military and magical advice. He seeks to maintain his independence while supporting the Grizzlies' mission of protecting Quickstone from threats. He wishes to find ways to better manage the town's resources, improve its defenses, and help improve the lives of its inhabitants.

Personality Traits:
+ Punctual and organized
+ Loyal to his friends and Breland
+ Pragmatic and practical
+ Determined to prevent future wars
+ Slightly arrogant about his magical abilities

Secrets:
+ Jones suffers from a rare ailment called "Cantrip Fever," which causes him to spontaneously manifest spells without concentration. It is a closely-guarded secret, as many believe it to be a sign of divine favor.
+ During the war, Jones had an affair with a high-ranking Karrnathi officer, which resulted in a child. The child was sent to a Karrnathi orphanage and is believed to be dead.

Relevant D&D 5e Skill Checks:
+ A Perception check might reveal Jones' unusual aura when he is casting spells.
+ An Investigation check could uncover his hidden affair if the characters delve into Jones' past.
+ A Persuasion check might help determine whether Jones is willing to share hisCharacter Name: Cameron Calamity Jones

Race: Human

Class: Sorcerer (Draconic Bloodline - Bronze Dragon) - Level 5

Attributes:
- Strength: 12 (+1)
- Dexterity: 14 (+2)
- Constitution: 14 (+2)
- Intelligence: 16 (+3)
- Wisdom: 12 (+1)
- Charisma: 18 (+4)

Proficiencies:
- Saving Throws: Wisdom, Charisma
- Skills: Arcana, History, Investigation, Persuasion

Proficiency Bonus: +3

Hit Points: 48 (10 from Constitution, 38 from Class)

Weapons and Equipment:
- Quarterstaff (Proficient) - Attack Bonus: +5
- Dagger (Proficient) - Attack Bonus: +4
- Light Crossbow (Proficient) - Attack Bonus: +3
- Scale Mail Armor - AC: 14 (+2 Dex)
- Explorer's Pack
- Dungeoneer's Pack
- Healing Potion (x3)
- Potion of Cure Wounds (x2)
- Potion of Invisibility
- Potion of Fire Breath (Draconic Ancestry)
- A set of traveler's clothes
- A map of the Five Nations
- A small, ornate dragon figurine (personal item)

Class Features:
- Cantrips: Acid Splash, Mage Hand, Prestidigitation
- Spell Slots: 5 level 1 slots, 4 level 2 slots
- Draconic Ancestry: Resistance to acid and fire damage, ability to cast Fire Breath once per long rest
- Font of Magic: Once per long rest, Jones can regain all expended sorcery points
- Metamagic: Quickened Spell

Class and Level: Sorcerer 5

Armor Class: 16 (Scale Mail Armor, Dexterity)

Spells:
Level 1: Burning Hands, Chromatic Orb, Detect Magic, Disguise Self, Fireball, Mage Armor, Magic Missile, Shield
Level 2: Darkness, Hold Person, Invisibility, Scorching Ray, Web

Inventory:
- 50 gold pieces
- A letter from a former soldier, expressing gratitude for Jones' leadership during the war
- A small locket containing a picture of his child (secret)
- A set of fine clothes for formal occasions
- A military medal awarded for bravery in battle
- A journal detailing his experiences during the war (personal)
- A map of Breland, marked with potential sites for rebuilding efforts
- A set of common clothes for everyday wear
- A set of traveler's clothes
- A small pouch containing 15 gold pieces

Long-term Goals: Rebuild Breland and prevent future wars.
Immediate Wants and Needs: Maintain independence while supporting the Grizzlies' mission. Improve Quickstone's resources, defenses, and living conditions.
Personality Traits: Punctual, organized, loyal, pragmatic, determined, slightly arrogant.
Secrets: Suffers from Cantrip Fever,




In [190]:
response = ""
async for chunk in supervisor.respond("Create a Brelish war veteran"):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n') + '\n\n\n'))

**Character Name:** Cameron Calamity Jones

**Backstory:** Cameron Jones was born and raised in the bustling city of Sharn, Breland. Growing up, he was always drawn to the arcane and the magical, fascinated by the wandslingers, the new breed of soldiers who could harness magic on the battlefield. When the Last War broke out, he saw an opportunity to serve his nation and follow his passion. He enlisted in the Brelish army and was trained as a wandslinger, quickly rising through the ranks thanks to his natural aptitude for arcane combat.

During the war, Cameron distinguished himself with acts of bravery and tactical genius. His unit, the Lion's Roar, became known for their daring charges and fearsome power on the battlefield. However, the war took its toll on Cameron, both physically and emotionally. He lost many close comrades, and the horrors of war haunt his dreams to this day.

At the end of the war, Cameron was honorably discharged and returned to Sharn. He initially struggled to adapt to civilian life, feeling lost without the structure and purpose that the army provided. But his love for magic and his desire to help his nation never waned. He used his skills and connections to become a renowned wandslinger for hire, often taking on dangerous jobs that others would shun.

**Long-term Goals:** Cameron's long-term goal is to prove that wandslingers are more than just battlefield warriors. He hopes to use his skills to help rebuild Breland and make it a safer, more prosperous place. He also aspires to establish a school or academy for young wandslingers, teaching them the arts of arcane combat and instilling in them a sense of duty and honor.

**Immediate Wants and Needs:** In the short term, Cameron is seeking to establish himself as a reliable and highly skilled wandslinger for hire. He needs to find steady work that allows him to pay his bills and maintain his equipment. He is also looking to reconnect with old comrades from the Lion's Roar, to share stories and reminisce about the good old days.

**Personality Traits:** Cameron is fiercely loyal to Breland and its people. He is brave, resolute, and unyielding in the**Character Name:** Cameron Calamity Jones

**Race:** Human (Variant: Tiefling)

**Class:** Wandslinger (Homebrew Class) - Level 5

**Attributes:**
- Strength: 14
- Dexterity: 16 (Dexterity Modifier: +3)
- Constitution: 14
- Intelligence: 16
- Wisdom: 12
- Charisma: 14

**Proficiencies:**
- Armor Proficiency: Light armor, medium armor, shields
- Weapon Proficiency: Simple weapons, martial weapons, firearms
- Tool Proficiency: Artisan's tools (blacksmith's tools), thieves' tools
- Skill Proficiency: Athletics, Acrobatics, Arcana, History, Investigation, Perception, Stealth

**Proficiency Bonus:** +3

**Saving Throws:** Intelligence, Dexterity

**Feats and Proficiencies:**
- Shield Master (Feat)
- Wandslinger's Focus (Class Feature)

**Hit Points:** 52 (10 HP from level 5 Wandslinger, 42 HP from 14 CON)

**Weapons and Equipment:**
- Rapier (Longsword for melee combat when using a shield)
- Shield
- Wand (Quarterstaff for ranged combat)
- 20 Bullets (for firearm)
- Explosive Runes (5)
- Arcane Focus (Quarterstaff)
- Leather Armor
- Adventurer's Pack
- Explorer's Pack
- 50 Gold Pieces
- A small, intricately crafted silver locket containing a picture of the Lion's Roar unit
- A set of custom-made blacksmith's tools, a gift from a comrade

**Attack Bonus:**
- Rapier: +6 to hit, 9 (1d8 + 4) slashing damage
- Longsword: +6 to hit, 11 (1d10 + 4) slashing damage
- Shield: +2 AC bonus
- Wand (Quarterstaff): +6 to hit, 15 (4d6) force damage (or 1d10 + 4 force damage if using Explosive Runes)

**Class and Level:** Wandslinger 5

**Armor Class:** 16 (13 + 3 Dexterity Modifier)

**Spellcasting Ability:** Arcane Tradition (Homebrew Subclass) - Intelligence

**Spells:**
- Cantrips: Arcane Charge, Mage Hand, Mending, Message, Prestidigitation, Thaumaturgy
- 1st Level: Burning Hands, Cure Wounds, Detect Magic, Magic Missile, Shield
- 2nd Level: Darkvision, Enhance Ability, Hold Person, Scorching Ray, Web

**Inventory:**
- A set of fine clothes for formal occasions
- A map of Breland, marked with the locations of old comrades from the Lion's Roar
- A journal to document his adventures and experiences
- A set of tools for repairing and modifying his wands and equipment
- A small, enchanted amulet that grants

In [171]:
response = ""
async for chunk in supervisor.respond("Create a Brelish war veteran"):
    response += chunk
    clear_output()
    display(Markdown(response.replace('\\n', '\n')))

Backstory:
Cameron Calamity Jones is a Brelish war veteran who served as a wandslinger during the Last War. Originally a non-combatant with a background in arcane studies, Cameron was drafted into the Brelish military and trained to wield a wand in combat. He quickly rose through the ranks, earning a reputation as a formidable and relentless combatant, earning him the nickname "Calamity" from his peers. After the war, Cameron returned to Breland and settled in Quickstone, taking up a position as a sheriff's deputy and using his skills as a wandslinger to maintain law and order in the town and surrounding areas.

Long-term Goals:
Cameron's long-term goal is to one day be appointed as the sheriff of Quickstone and build a strong and respected force to protect the town and its citizens. He also hopes to build a peaceful and prosperous future for Breland, and to help the nation rebuild after the destruction caused by the Last War.

Immediate Wants and Needs:
Cameron is currently focused on maintaining law and order in Quickstone, and is always on the lookout for troublemakers and criminals who threaten the peace. He is also in constant need of supplies and equipment to maintain his wand and keep it functioning properly, as well as medical supplies to treat any injuries sustained during his duties.

Personality Traits:
Cameron is a disciplined and driven individual, with a strong sense of duty to his country and his community. He is also fiercely independent and stubborn, and is not afraid to stand up for what he believes in, even if it means going against authority. He is deeply protective of those he cares about, and will stop at nothing to protect them from harm.

Secrets:
Cameron has a dark secret from his time in the war: he was responsible for the death of a fellow soldier who had threatened to reveal a scandal involving a high-ranking officer. Cameron has spent years living with the guilt of this act, and is haunted by the memory of his former comrade. He has also been secretly training a young Brelish soldier, believing that he can help the young man reach his full potential and become a valuable asset to the Brelish military.

Relevant D&D 5e Skill Checks:
Players may be able to notice Cameron's proficiency with Smith's Tools, as he is constantly maintaining and repairing his wand. They may also be able to spot that he is a wandslinger by observing his wand at rest on his belt, or by witnessing him casting spells during combat. With a successful Investigation check, players may be able to discover Cameron's dark secret, or overhear him talking to his secret trainee. With a successful Persuasion check, players may be able to convince Cameron to look the other way in certain situations, or to provide them with information or assistance.Character Name: Cameron Calamity Jones

Race: Human (Variant Human)

Class: Wandslinger (Homebrew Class) - Level 3

Attributes:
- Strength: 12 (+1)
- Dexterity: 16 (+3)
- Constitution: 14 (+2)
- Intelligence: 16 (+3)
- Wisdom: 12 (+1)
- Charisma: 14 (+2)

Proficiencies:
- Armor Proficiency: Light Armor
- Weapon Proficiency: Simple Weapons, Martial Weapons, Wands
- Tool Proficiency: Smith's Tools
- Skill Proficiency: Athletics, Investigation, Persuasion, Perception

Proficiency Bonus: +3

Saving Throws: Wisdom, Charisma

Feats and Proficiencies:
- Feat: Sharpshooter (Dexterity)
- Proficiency: Resilient (Constitution)

Hit Points: 39 (3d8 for Human + 3d10 for Wandslinger)

Weapons and Equipment:
- Wand of Fireball (3 charges)
- Wand of Magic Missile (4 charges)
- Wand of Cure Wounds (2 charges)
- Short Sword (proficient)
- Leather Armor (armor class 11 + Dexterity modifier)
- Explorer's Pack
- Smith's Tools
- A set of traveler's clothes
- A set of Brelish military uniform (for formal occasions)
- A small pouch containing 15 gold pieces
- A locket with a picture of his family (personal item)

Attack Bonus:
- Short Sword: +5 (Dexterity modifier + Proficiency Bonus)

Class and Level Features:
- Wandslinger: Wand Mastery, Wand Channeling, Wand Specialization (Fire)
- Level 3: 3rd-level Spell Slots (2 slots of 3rd level)

Spells:
- Cantrips: Mage Hand, Prestidigitation, Thaumaturgy
- 1st Level: Burning Hands, Chromatic Orb, Cure Wounds, Detect Magic, Shield, Thunderwave
- 2nd Level: Darkness, Enhance Ability, Hold Person, Scorching Ray, Web
- 3rd Level: Fireball, Fly, Hypnotic Pattern, Lightning Bolt, Vampiric Touch

Armor Class: 14 (Leather Armor + Dexterity modifier)

Inventory:
- A set of spare wand components (for crafting new wands)
- A spellbook containing spells from the Wandslinger class
- A set of lockpicks (personal item)
- A map of Breland (personal item)
- A letter from a former comrade (personal item)
- A small pouch containing 50 gold pieces (for supplies and equipment)
- A set of manacles (for apprehending criminals

In [ ]:
q = "Tell me about the languages of Eberron."

In [206]:
%%time
for chunk in supervisor.respond("Tell me about the languages of Eberron."):
    display(chunk)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'lookup'

CPU times: user 982 ms, sys: 169 ms, total: 1.15 s
Wall time: 1.15 s


In [207]:
%%time
for chunk in supervisor.respond("Create a House Cannith item."):
    display(chunk)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'character'

CPU times: user 742 ms, sys: 146 ms, total: 888 ms
Wall time: 886 ms


In [ ]:
# TODO: class ShortResponseAgent(Agent)
# TODO: class CharacterCreatorAgent(Agent)
# TODO: class CharacterDetailerAgent(Agent)
# TODO: class OriginalContentAgent(Agent)

In [35]:
import networkx

In [94]:
q = "Tell me about fashion in the five nations."
response = get_info(q)
print(response)

[2025-01-20T04:24:22Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:24:22Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:24:22Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:24:22Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. During the Last War, the fashion styles of the Five Nations evolved to distinguish their soldiers and improve their tools of war. Common soldiers typically wore light, medium, or heavy armor made of leather, metal, or a combination of both.

    2. Elite forces, officers, and mercenaries used different styles and materials. Each nation had its own distinct approach to fashion, both in its armor and civilian clothing.

    3. Beyond practical considerations, adventurers might choose to wear specific clothing as a cultural expression. For example, a Cyran might wear gloves and masks, a Karrn might wear gazyrs sewn onto leather armor, an Aundairian might wear a cloak with strange proportions, a Thrane might wear clothes covered in embroidered patches, an Aundairian might wear clothes covered in embroidered patches, and a Brel might wear pointy-toed shoes and long puffy sleeves.

    4. Many contemporary fashion designers now produce lines of fashion specifically for adventurers.

    5

In [97]:
q = "Tell me about fashion in the five nations."
response = get_info(q)
print(response)

[2025-01-20T04:25:16Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:25:16Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:25:16Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:25:16Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. The common armor of today for the Five Nations include light, medium, and heavy armor. These typically involve a metal helmet, breastplate for medium armor, and leather or chainmail for light armor. However, each nation has its distinct approach to fashion both in its armor and civilian clothing.

    2. Adventurers from the Five Nations often represent their homeland in their clothing, although not always for practicality. Adventurers are an important part of each nation, with adventuring guilds and independents playing crucial roles in their cultures.

    3. The Church of the Silver Flame, one of the most significant religions in the Five Nations, is recognized by its delicately embroidered, silver-dyed denim robes. Their vestments are sewn with extra space for mobility, and are usually closed with toggles rather than fasteners.

    4. In Middle Tavick's Landing, one can find fusion of Brelish lapels, Thrane cloaks, and other styles, as it serves as the melting pot of cultures i

In [99]:
q = "Suggest an outfit for a dragonmarked heir."
response = get_info(q)
print(response)

[2025-01-20T04:26:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:26:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:26:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:26:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. Light Armor: Because the dragonmark heir does not have proficiency in heavy armor, a light armor will offer better protection while being able to move easily. Many Mror wear decorative armor that uses the statistics of light armor, but evokes the general flavor of a heavier breastplate. This will allow the heir to retain the martial aspect to their attire that the Mror generally favor, but still be able to move freely.

    2. Robe or Tunic: To balance out the armor, a long robe or tunic can be used as the top garment. This will not only provide a more elegant look, but also offer more coverage for the dragonmark on their arm/chest.

    3. Sash or Belt: A sash or belt can be worn around the waist to hold up the robe/tunic and also add a bit of color and personality to the outfit.

    4. Boots: Boots would be a practical choice for footwear, as they offer protection and support for the feet while moving.

    5. Accessories: To complete the outfit, the heir could wear various acces

In [100]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 10967 MiB, 4129 MiB


In [36]:
def get_one_word_answer(q: str) -> str:
    results = retrieve(q, embedding_model, 5, 'cosine')

    retrieved_text = dedent("""
    -----
    """).join(results['text'])

    prompt = f"""
    [INST]
    Use the following information (until the final cutoff =====) to answer the user query Q below.
    The answer has to be one word only.
    You are trying to get the most accurate answer in one shot.

    {retrieved_text}

    =====

    Q:
    {q}

    A:
    [/INST]
    """
    print(len(prompt))

    model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    generated_ids = model.generate(**model_inputs, 
                                   max_length=50,
                                   max_new_tokens=256, 
                                   do_sample=True, 
                                   eos_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return (response.split('A:\n')[1] if 'A:\n' in response else response).strip()


In [37]:
q = "What is the name of the Breland veterans community in Quickstone?"
response = get_one_word_answer(q)
print(response)

[2025-01-21T01:40:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:40:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:40:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:40:31Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` w

8460
Tents


In [ ]:
q_and_a = [
    ("What is the name of the prominent university in Sharn?", "Morgrave"),
    ("What type of magic does Karrnath use in its military?", "Necromancy"),
    ("Who leads Kind's Dark Lanterns?", "Vron"),
    ("Who leads King's Dark Lanterns?", "Vron"),
    ("What is the name of the Breland veterans community in Quickstone?", "Tents"),
]

In [38]:
# Required globals: model, embedding_model, tokenizer, retreive. I might also need a score or k or other parameters.
retrieve = retrieve_baseline
def generate_character(q: str) -> str:
    results = retrieve(q, embedding_model, 10, 'cosine')

    retrieved_text = dedent("""
    -----
    """).join(results['text'])

    prompt = f"""
    [INST]
    Use the following information (until the final cutoff =====) to create a character based on the 5th Edition rules for the world of Eberron.
    Use the prompt P as a starting point.
    Give the complete character sheet.
    A complete character sheet should include name, race, class, level, attributes, proficiencies based on the class and fifth edition rules, 
    proficiency bonus based on class and level, saving throws bonus based on class, feats and proficiency , hit points based on class and level, 
    weapons and equipment, attack bonus for each weapon based on abilities, any race or class based features, class and level, attack roll, 
    and equipment, armor class based on dexterity and armor, and if the character has spellcasting ability, spells based on class and level, 
    as well as the number of spell slots. 
    Include an inventory based on their level, class, and race, with a few personal touches in the items.
    
    Also include a compelling and detailed backstory of two paragraphs related to the world of Eberron. 
    This background should include their nation.
    Include another short paragraph about their intentions.
    Include one other short paragraph about their tactics in combat.
    Finally, include one paragraph for the visuals.

    {retrieved_text}

    =====

    Prompt P:
    {q}

    Character:
    [/INST]
    """

    model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    generated_ids = model.generate(**model_inputs, 
                                   max_new_tokens=2048, 
                                   do_sample=True, 
                                   eos_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return (response.split('Character:\n')[1] if 'Character:\n' in response else response).strip()

    

In [103]:
character = generate_character("War veteran")
print(character)

[2025-01-20T04:34:02Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:34:02Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:34:02Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:34:02Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


8. Character: Caelum "Cal" Iarwain
    9. Race: Human
    10. Class: Ranger 1
    11. Attributes (5e standard array): Str 12, Dex 14, Con 10, Int 8, Wis 12, Cha 10
    11. Class Abilities:
        1. Starting Proficiencies: Perception, Survival
        2. Level 1 Fighting Style: Archery
        3. Ranger Spells: Cat's Grace, Hunter's Mark
        4. Favored Enemy: Orcs
        5. Natural Explorer: Swamp, Urban
        6. Skills: Climb, Dexterity, Intelligence, Investigation, Nature, Navigation, Survival, Persuasion
        7. Tools: None
    12. Proficiency Bonus (5e default): +2
    13. Saving Throws:
        1. Wisdom (Race)
        2. Dexterity (Class)
    14. Feats & Levels: None
    15. Hit Points: 10 (Levels: 1 Ranger) + Hit Die (next level: 1d10)
    16. Weapons & Equipment:
        1. Shortbow
        2. Quiver (20 arrows)
        3. Short Sword
        4. Leather Armor
    17. Attack Bonus (Per Stat):
        1. Shortbow: +5 to hit (1d8 damage)
        2. Short Sword: +5 to hi

In [40]:
character = generate_character("Comin relief, similar to C3p0")
print(character)

[2025-01-21T01:43:24Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:43:24Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:43:24Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:43:24Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Name: RedLight Disiuct p
     Race: Golem (Grey Iron)
     Class: Sorcerer (Harbinger of Faith)
     Level: 5
     Attributes:
       Strength: 14 (+2)
       Dexterity: 12 (+1)
       Constitution: 14 (+2)
       Intelligence: 16 (+3)
       Wisdom: 8 (-1)
       Charisma: 18 (+4)
     Proficiencies: Arcana, History, Insight, Religion
     Proficiency Bonus: +3
     Saving Throws: Wisdom (+0), Charisma (+6)
     Feats and proficiencies:
       - Mage Armor (1 level sorcerer)
       - Spellcasting (5 levels sorcerer)
     Hit Points: 37 (5d8 for Golem + 21 (5d6 + 5 from sorcerer levels))
     Weapons and equipment:
       - Dagger (attack roll +5)
       - Quarterstaff (attack roll +4)
       - Tindertwig (no attack bonus, helps with igniting objects)
       - A necklace with a charm of healing (1d4 + 2 HPs)
       - A pouch containing 2d4 sprigs of Pacifycus, used to reduce fevers
       - A small wooden effigy of Sovereign Host, used as an additional focus in spells
     Armor Class:

In [39]:
character = generate_character("War veteran 3rd level")
print(character)

[2025-01-21T01:41:52Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:41:52Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:41:52Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-21T01:41:52Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Name: Cameron Calamity Jones
     Race: Human
     Class: Wandslinger (Blaster)
     Level: 3

    Attributes:
     - Strength (Str): 10 (+0)
     - Dexterity (Dex): 14 (+2)
     - Constitution (Con): -1 (-2)
     - Intelligence (Int): 14 (+2)
     - Wisdom (Wis): 12 (+1)
     - Charisma ( Cha): 10 (+0)

    Proficiencies:
     - Armor: Light armor, medium armor, shields
     - Weapons: Simple weapons, martial weapons
     - Skills: Arcana +4, Athletics +2, Medicine +3

    Proficiency Bonus: +2
     Saving Throws Bonus: Dex +4, Con +4

    Hit Points: 39 (6d10 + 12)

    Equipment:
     - Chain Shirt
     - Heavy Crossbow with 20 bolts
     - Two Longswords
     - Studded Leather Armor
     - Dungeoneer's pack
     - Leather armor, shield, and 14 bolts (equipped)
     - A wand (your magical focus) that can cast spells from the Enchantment and Evocation spell lists

    Attack Bonus:
     - Longsword: +4
     - Heavy Crossbow: +3

    Class Features:
     - Blaster: Your choice grants 

In [108]:
character = generate_character("5th level gunslinger")
print(character)

[2025-01-20T04:42:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:42:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:42:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
[2025-01-20T04:42:21Z WARN  lance_core::utils::tokio] Number of CPUs is less than or equal to the number of IO core reservations. This is not a supported configuration. using 1 CPU for compute intensive tasks.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


5th-level Wandslinger (Blaster)
    Medium humanoid (human)
    Str 10, Dex 14, Con 14, Int 16, Wis 12, Cha 10,
    Saving throws Dex +5, Con +5
    Skills Arcana +6, Athletics +3, Medicine +4
    Hit Dice: 5d10
    Proficiencies: Light armor, Medium armor, Shields; Simple weapons, Martial weapons
    Senses: passive Perception 11
    Languages: Common, Dwarvish

    Attributes:
    Attack bonus (longbow): +7
    Armor Class: 17

    Special Features:
    True Shot (Gunslinger): You can take a −4 penalty to the attack roll to gain a +10 bonus to the damage roll for that attack.
    Assess Foe (Gunslinger): By taking a minute of time to examine your adversary, you can learn information about the adversary and make special moves to gain an advantage in battle.
    Magic Ammunition (Gunslinger): You learned to cast spells as a bonus action without using a spell slot and can now imbue your quiver of arrows with a magical power. When you activate this power, select one spell from the list b

# Test Langchain

In [21]:
!pip install langchain-huggingface --upgrade

In [25]:
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core import messages

In [23]:
!nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 4087 MiB, 11009 MiB


['AIMessage',
 'AIMessageChunk',
 'AnyMessage',
 'BaseMessage',
 'BaseMessageChunk',
 'ChatMessage',
 'ChatMessageChunk',
 'FunctionMessage',
 'FunctionMessageChunk',
 'HumanMessage',
 'HumanMessageChunk',
 'InvalidToolCall',
 'MessageLikeRepresentation',
 'RemoveMessage',
 'SystemMessage',
 'SystemMessageChunk',
 'ToolCall',
 'ToolCallChunk',
 'ToolMessage',
 'ToolMessageChunk',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_message_from_dict',
 'ai',
 'base',
 'chat',
 'convert_to_messages',
 'convert_to_openai_messages',
 'filter_messages',
 'function',
 'get_buffer_string',
 'human',
 'merge_content',
 'merge_message_runs',
 'message_chunk_to_message',
 'message_to_dict',
 'messages_from_dict',
 'messages_to_dict',
 'modifier',
 'system',
 'tool',
 'trim_messages',
 'utils']

In [ ]:

character_prompter = HuggingFacePipeline.from_model_id(
    model_id=model_path,
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
    ),
)



In [ ]:

character_generator = HuggingFacePipeline.from_model_id(
    model_id=model_path,
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
    ),
)



In [ ]:
# from langchain_huggingface.llms import HuggingFacePipeline
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# model_id = "gpt2"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
# hf = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# from langchain_core.prompts import PromptTemplate

# template = """Question: {question}

# Answer: Let's think step by step."""
# prompt = PromptTemplate.from_template(template)

# chain = prompt | hf

# question = "What is electroencephalography?"

# print(chain.invoke({"question": question}))